In [1]:
# Setting up
!pip install transformers
!pip install transformers[sentencepiece]
!pip install datasets

     |████████████████████████████████| 2.8 MB 2.7 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 3.3 MB 19.3 MB/s 
     |████████████████████████████████| 636 kB 46.0 MB/s 
     |████████████████████████████████| 895 kB 43.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.1 MB 4.1 MB/s 
     |████████████████████████████████| 270 kB 4.3 MB/s 
     |████████████████████████████████| 243 kB 42.3 MB/s 
     |████████████████████████████████| 123 kB 47.0 MB/s 
     |████████████████████████████████| 1.3 MB 41.4 MB/s 
     |████████████████████████████████| 294 kB 51.8 MB/s 
     |████████████████████████████████| 142 kB 55.8 MB/s 


# Fine-tuning a model with a Trainer API

HuggingFace Transformers는 Trainer class를 제공한다.

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### 이전 섹션과 동일한 예제

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Downloading: 0.00B [00:00, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

...

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

## Training
Trainer를 정의하기 전 첫 번째 단계는 training과 evaluation에서 사용될 Trainer의 모든 hyperparameters를 갖는 TrainingArguments class를 정의하는 것이다.

In [3]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer") # 어느 directory 저장할지 (나머지는 default)

두 번째 단계는 model을 정의하는 것.

In [4]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# 저번과 다르게 warning이 발생하는 이유는
# BERT가 sentence 쌍을 분류하는 데 pretrain 되어 있지 않기 때문.
# 그래서 pretrained model의 head가 버려지고 sentence classification에 맞는 새로운 head가 대신 추가됨.

# warning은 일부 가중치(버려진 pretraining head에 해당하는 가중치)가 사용되지 않았고,
# 일부 다른 가중치(새로운 head의 가중치)가 random하게 초기화되었음을 나타냄.
# 모델을 train하라고 권장 -> 우리가 하려는 fine-tuning

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

그럼 이제 Trainer를 정의해보자.

In [5]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [6]:
# fine-tuning 하려면 그냥 Trainer의 train method를 사용하면 됨

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1377


Step,Training Loss
500,0.484700
1000,0.241900


Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1000
Configuration saved in test-trainer/checkpoint-1000/config.json
Model weights saved in test-trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1377, training_loss=0.28837688227191555, metrics={'train_runtime': 384.3256, 'train_samples_per_second': 28.632, 'train_steps_per_second': 3.583, 'total_flos': 405470580750720.0, 'train_loss': 0.28837688227191555, 'epoch': 3.0})

500 step마다 training loss를 보여주지만, 아래와 같은 이유로 모델 performance에 대해서는 report 안 해줌.
- Trainer에 evaluation_strategy를 설정 안해줘서. (steps는 매 eval_steps마다 evaluate, epoch는 각 epoch이 끝날 때마다 evaluate)
- evaluation 할 때 compute_metrics로 metric을 지정해주지 않아서.

## Evaluation
다음 train 때 사용할 유용한 compute_metrics function을 만들어보자.
- compute_metrics function은 EvalPrediction object를 가져야 함. (EvalPrediction object는 predictions field와 label_ids field를 갖는 named tuple.)
- compute_metrics function은 string을 float으로 mapping 하는 dictionary를 return. string은 return될 metric의 이름, floats는 그 값.

In [7]:
# 모델로부터 predictions를 얻기 위해 Trainer.predict를 사용할 수 있음
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Prediction *****
  Num examples = 408
  Batch size = 8


(408, 2) (408,)


predict method의 아웃풋은 세 개의 field (predictions, label_ids, metrics)를 갖는 다른 named tuple임.
- metrics field는 loss와 predict 하는 데 걸린 시간 같은 time metric을 포함함.
- predictions는 408(validation dataset의 sample 개수) x 2 모양의 이차원 배열임. 2는 각 label에 해당하는 logits.

In [8]:
import numpy as np
preds = np.argmax(predictions.predictions, axis=-1) # 두 번째 axis의 값이 최댓값인 index를 가져옴

이제 preds를 label과 비교할 수 있게 됐다. huggingface datasets library에서 제공되는 compute_metric function을 사용해보자.

In [9]:
from datasets import load_metric

metric = load_metric("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

{'accuracy': 0.8578431372549019, 'f1': 0.8986013986013986}

위 과정들을 하나로 Wrapping 해보자.

In [10]:
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

compute_metrics function을 포함한 Trainer를 새로 정의해보자.

In [11]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "

In [12]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1377


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.557327,0.806373,0.872375
2,0.552700,0.458040,0.862745,0.903448
3,0.333900,0.560826,0.867647,0.907850


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 8
Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 8
Saving model checkpoint to test-trainer/checkpoint-1000
Configuration saved in test-trainer/checkpoint-1000/config.json
Model weights saved in test-trainer/checkpoin

TrainOutput(global_step=1377, training_loss=0.37862846690325436, metrics={'train_runtime': 406.344, 'train_samples_per_second': 27.081, 'train_steps_per_second': 3.389, 'total_flos': 405470580750720.0, 'train_loss': 0.37862846690325436, 'epoch': 3.0})